In [2]:
#%matplotlib nbagg

import geofield
import dipole, fibonacci_sphere
import scipy, numpy
import newleg
import xyzfield
from matplotlib import pyplot, rcParams
import matplotlib
rcParams.update({'font.family': "Source Sans Pro"})
rcParams.update({'font.size': 14})

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import bootstrap
import importlib; importlib.reload(bootstrap)

<module 'bootstrap' from '/home/yo/Programs/python/geofieldpy/bootstrap.py'>

In [4]:
sha=geofield.SHADIFData("../data/sha.dif.14k/coef_EPSL_tnmgh.dat", "../data/sha.dif.14k/error_coef_EPSL_tnmgh.dat")

In [4]:
(x,y,z), (x_dev, y_dev, z_dev)=bootstrap.bootstrap(dipole.ecc_dipole, (sha.gcomp[... , :8],), errors=(sha.gcomp_err[..., :8],))

In [5]:
pyplot.show(pyplot.plot(sha.years, z,
                        sha.years, z+z_dev,
                        sha.years, z-z_dev))

In [4]:
def szrphi(x,y,z):
    s, zz, phi = dipole.xyz2cyl(x,y,z)
    r = numpy.sqrt(x**2 + y**2 + z**2)
    return s, zz, r, phi

#wow this is cool as heck
bigfunc = lambda g: szrphi(*dipole.ecc_dipole(g))

In [42]:
(s,z,r, phi), (s_dev, z_dev, r_dev, phi_dev) = bootstrap.bootstrap(bigfunc, (sha.gcomp[..., :8],), errors=(sha.gcomp_err[..., :8],))

In [5]:
fig, ax = pyplot.subplots(figsize=(20,10))

ax.plot(sha.years, s, color="green", label="s")
ax.plot(sha.years, z, "m-", label="z")
ax.plot(sha.years, r, "b-", label="r")

ax.fill_between(sha.years, r-r_dev, r+r_dev, color="blue", lw=0, alpha=0.07)
ax.fill_between(sha.years, z-z_dev, z+z_dev, color="magenta", lw=0, alpha=0.07)
ax.fill_between(sha.years, s-s_dev, s+s_dev, color="green", lw=0, alpha=0.07)

ax.set_xlabel("Years AD")
ax.set_ylabel("s,z,r (km)")
ax.legend()

ax.set_xbound(-1000,sha.years[-1])

pyplot.show(fig)

NameError: name 's' is not defined

In [174]:
fig, (ax1, ax2) = pyplot.subplots(2,1)


ax1.plot(sha.years, nlon_1, "b-", label="Geomagnetic pole $(l=1)$")
ax1.plot(sha.years, (nlon_1+nlon_1_dev), "b--")
ax1.plot(sha.years, (nlon_1-nlon_1_dev), "b--")

ax2.plot(sha.years, nlat_1, "b-")
ax2.plot(sha.years, nlat_1+nlat_1_dev, "b--")
ax2.plot(sha.years, nlat_1-nlat_1_dev, "b--")

ax1.plot(sha.years, nlon_2, "r-", label="Magnetic pole $(l=2)$")
ax1.plot(sha.years, (nlon_2+nlon_2_dev), "r--")
ax1.plot(sha.years, (nlon_2-nlon_2_dev), "r--")

ax2.plot(sha.years, nlat_2, "r-")
ax2.plot(sha.years, nlat_2+nlat_2_dev, "r--")
ax2.plot(sha.years, nlat_2-nlat_2_dev, "r--")

ax1.plot(sha.years, nlon_4, "g-", label="Magnetic pole $(l=4)$")
ax1.plot(sha.years, (nlon_4+nlon_4_dev), "g--")
ax1.plot(sha.years, (nlon_4-nlon_4_dev), "g--")

ax2.plot(sha.years, nlat_4, "g-")
ax2.plot(sha.years, nlat_4+nlat_4_dev, "g--")
ax2.plot(sha.years, nlat_4-nlat_4_dev, "g--")

ax1.plot(sha.years, nlon_10, "k-", label="Magnetic pole $(l=10)$")
ax1.plot(sha.years, (nlon_10+nlon_10_dev), "k--")
ax1.plot(sha.years, (nlon_10-nlon_10_dev), "k--")

ax2.plot(sha.years, nlat_10, "k-")
ax2.plot(sha.years, nlat_10+nlat_10_dev, "k--")
ax2.plot(sha.years, nlat_10-nlat_10_dev, "k--")

ax1.legend(loc="upper left")

ax1.set_ylabel("Pole longitude (º)")
ax2.set_ylabel("Pole latitude (º)")
ax2.set_xlabel("Years AD")

pyplot.show(fig)

In [ ]:
from joblib import Parallel, delayed

#be carfeful my man
def boot_it(order):
    return bootstrap.bootstrap(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=1, order=order, south=False)

orders = (1, 2, 4, 10)
(((nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev)),
 ((nlon_2, nlat_2), (nlon_2_dev, nlat_2_dev)),
 ((nlon_4, nlat_4), (nlon_4_dev, nlat_4_dev)),
 ((nlon_10, nlat_10), (nlon_10_dev, nlat_10_dev))) = Parallel(n_jobs=4)(delayed(boot_it)(o) for o in orders)

### get urself 2400 coffees before hitting shift+enter my dude

In [66]:
#hack bootstrap to use minimal differences on the stdev calculation? done 

((nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), stdev_func=bootstrap.stdev_cyclic, errors=(sha.gcomp_err,), iterations=300, n_jobs=4, order=1, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_1.txt", numpy.array((nlon_1, nlat_1, nlon_1_dev, nlat_1_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")
((nlon_2, nlat_2), (nlon_2_dev, nlat_2_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), stdev_func=bootstrap.stdev_cyclic, errors=(sha.gcomp_err,), iterations=300, n_jobs=4, order=2, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_2.txt", numpy.array((nlon_2, nlat_2, nlon_2_dev, nlat_2_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")
((nlon_4, nlat_4), (nlon_4_dev, nlat_4_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), stdev_func=bootstrap.stdev_cyclic, errors=(sha.gcomp_err,), iterations=300, n_jobs=4, order=4, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_4.txt", numpy.array((nlon_4, nlat_4, nlon_4_dev, nlat_4_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")
((nlon_10, nlat_10), (nlon_10_dev, nlat_10_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), stdev_func=bootstrap.stdev_cyclic, errors=(sha.gcomp_err,), iterations=300, n_jobs=4, order=10, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_10.txt", numpy.array((nlon_10, nlat_10, nlon_10_dev, nlat_10_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")

2:09:29.133388 | done! (300 iterations)
0:37:31.599668 | iterations: 64 / 300

/home/josecper/Programs/python/xyzfield.py:389: RuntimeWarning: invalid value encountered in add
  x += rparamexp*(g*cossin)*dleg*(-scipy.sin(theta))


2:56:07.382940 | done! (300 iterations)
0:37:55.198103 | iterations: 56 / 300

/home/josecper/Programs/python/xyzfield.py:389: RuntimeWarning: invalid value encountered in add
  x += rparamexp*(g*cossin)*dleg*(-scipy.sin(theta))


0:40:33.911831 | iterations: 60 / 300

...

3:27:02.405857 | iterations: 224 / 300

/home/josecper/Programs/python/xyzfield.py:389: RuntimeWarning: invalid value encountered in add
  x += rparamexp*(g*cossin)*dleg*(-scipy.sin(theta))


4:11:15.803964 | iterations: 272 / 300

/home/josecper/Programs/python/xyzfield.py:389: RuntimeWarning: invalid value encountered in add
  x += rparamexp*(g*cossin)*dleg*(-scipy.sin(theta))


4:36:56.320569 | done! (300 iterations)


In [4]:
theta, phi = fibonacci_sphere.grid(4000)

def avg_intensity(gcomp):    
    intensities = numpy.empty_like(sha.years)
    npoints = len(phi)

    for i, year in enumerate(sha.years):
        x,y,z = xyzfield.xyzfieldv2(gcomp[i, :], phi, theta)
        dec,inc,f,h = xyzfield.xyz2difh(x,y,z)
        intensities[i]=(f.sum(axis=0)/npoints)
        
    return intensities

In [5]:
(intensities, intensities_dev) = bootstrap.parallelize(avg_intensity, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=400, n_jobs=8, debug=True)

0:45:21.142041 | iterations: 296 / 300

In [8]:
theta, phi = fibonacci_sphere.grid(4000)

def avg_vdm(gcomp):    
    avdm = numpy.empty_like(sha.years)
    npoints = len(phi)

    for i, year in enumerate(sha.years):
        x,y,z = xyzfield.xyzfieldv2(gcomp[i, :], phi, theta)
        dec,inc,f,h = xyzfield.xyz2difh(x,y,z)
        vdm = 10**-6/10**-7 * 6371000**3 / numpy.sqrt(1+3*numpy.cos(theta)**2) * f
        avdm[i]=(vdm.sum(axis=0)/npoints)
        
    return avdm

In [10]:
(avdm, avdm_dev) = bootstrap.parallelize(avg_vdm, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=128, n_jobs=8, debug=True)

0:28:13.113232 | done! (128 iterations)


In [11]:
def power_ratio(gcomp):
    p_dip=dipole.power(gcomp, lmin=1, lmax=1)
    p_quad=dipole.power(gcomp, lmin=2, lmax=2)
    return p_quad/p_dip

(pqpd, pqpd_dev) = bootstrap.parallelize(power_ratio, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=4000, n_jobs=8, debug=True)

0:00:08.508606 | iterations: 320 / 4000

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f48e2ad40d0>
Traceback (most recent call last):
  File "/usr/lib/python3.5/weakref.py", line 108, in remove
    def remove(wr, selfref=ref(self)):
KeyboardInterrupt


0:00:12.400288 | iterations: 464 / 4000

Process ForkPoolWorker-620:
Process ForkPoolWorker-623:
Process ForkPoolWorker-622:
Process ForkPoolWorker-619:
Process ForkPoolWorker-618:
Process ForkPoolWorker-621:
Process ForkPoolWorker-617:
Process ForkPoolWorker-624:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs

KeyboardInterrupt: 

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/site-packages/joblib/pool.py", line 357, in get
    racquire()
  File "/usr/lib/python3.5/site-packages/joblib/pool.py", line 357, in get
    racquire()
KeyboardInterrupt
  File "/usr/lib/python3.5/site-packages/joblib/pool.py", line 359, in get
    return recv()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/site-packages/joblib/pool.py", line 357, in get
    racquire()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)


In [11]:
fig, ax1 = pyplot.subplots()

ax1.plot(sha.years, intensities, "b-")
ax1.plot(sha.years, intensities+intensities_dev, "b--")
ax1.plot(sha.years, intensities-intensities_dev, "b--")

pyplot.show(fig)

In [9]:
numpy.savetxt("../data/pole_drift/pqpd.txt", numpy.array((pqpd, pqpd_dev)).T, header="#Quadrupole/dipole power ratio (ratio, stdev), dimensionless")

In [12]:
numpy.savetxt("../data/pole_drift/intensity.txt", numpy.array((intensities, intensities_dev)).T, header="#Average field intensity over 4000 points (intensity, stdev), uT")

In [64]:
((nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), stdev_func=bootstrap.stdev_cyclic, iterations=16, n_jobs=4, order=1, south=False, debug=True)

0:06:54.664227 | done! (16 iterations)


In [28]:
fig, ax1 = pyplot.subplots()

ax1.plot(sha.years, intensities, "b-")
ax1.plot(sha.years, intensities+intensities_dev, "b--")
ax1.plot(sha.years, intensities-intensities_dev, "b--")

pyplot.show(fig)

In [107]:
from mpl_toolkits.basemap import Basemap

base = Basemap(projection="npstere", lon_0 = 0, boundinglat=60)
years = numpy.array(sha.years)

In [108]:
fig, ax = pyplot.subplots()

base.drawcoastlines(ax=ax)
base.plot((nlon_1 % 360)[years >= -2000],
          (90-abs(90-nlat_1))[years >= -2000],
          latlon=True)
base.plot((nlon_1 % 360)[years >= -2000],
          (90-abs(90-nlat_1))[years >= -2000],
          latlon=True)

pyplot.show(fig)

In [40]:
nlon_1.shape

(279,)

In [106]:
fig, (ax1, ax2) = pyplot.subplots(2,1)

nlon_disc = nlon_1.copy() % 360

for slc in unlink_wrap(nlon_disc, thresh=0.60):
    ax1.plot(years[slc], nlon_disc[slc], "b-")

ax1.set_ybound(0, 360)

pyplot.show(fig)

In [61]:
years = numpy.array(sha.years)

nlon_10, nlat_10 = xyzfield.magnetic_north(sha.gcomp[years >= -2000], 10, south=False)
nlon_10, nlat_10 = numpy.array((nlon_10, nlat_10))

In [77]:
nlon_1, nlat_1 = xyzfield.magnetic_north(sha.gcomp[years >= -2000], 1, south=False)
nlon_1, nlat_1 = numpy.array((nlon_1, nlat_1))

In [63]:
from mpl_toolkits.basemap import Basemap
theta, phi = fibonacci_sphere.grid(40000)
maplat, maplon = 90 - numpy.rad2deg(theta), numpy.rad2deg(phi)
base = Basemap(projection="npstere", lon_0 = 0, boundinglat = 65)

In [45]:
for i, (year, gcomp) in enumerate(zip(years[years >= -2000], sha.gcomp[years >= -2000])):

    theta_pole, phi_pole = (numpy.pi/2 - numpy.deg2rad(nlat_10 % 360)[i], numpy.deg2rad(nlon_10 % 360)[i])
    xp,yp,zp=xyzfield.xyzfieldv2(gcomp, numpy.array([phi_pole]), numpy.array([theta_pole]), order=10, regular=False)

    difp, incp, intp, horp = xyzfield.xyz2difh(xp,yp,zp)

    threshold=0.92

    x, y, z = xyzfield.xyzfieldv2(gcomp, phi, theta, order=10)
    dif, inc, fnt, hor = xyzfield.xyz2difh(x,y,z)

    polar_inc = inc.copy()
    polar_inc[polar_inc < threshold*incp] = 0

    fig, ax = pyplot.subplots(figsize=(10,10))

    base.drawcoastlines(ax=ax)
    base.drawmeridians(numpy.arange(0, 375, 15), ax=ax)
    base.drawparallels(numpy.arange(-90, 100, 10), ax=ax)

    #base.contourf(maplon, maplat, polar_inc, latlon=True, tri=True, cmap="RdPu", vmin=threshold*incp, vmax=1.0*incp, levels=[0, *(numpy.linspace(threshold, 1.4, 100)*incp)])
    base.contourf(maplon, maplat, polar_inc, 250, latlon=True, tri=True, cmap="RdPu", vmin=threshold*incp)
    
    base.plot(nlon_10 % 360, 90 - numpy.abs(90 - nlat_10), latlon=True, color="black", linewidth=0.9, marker="o", markeredgewidth=0, markersize=5)
    base.scatter(nlon_10[i] % 360, 90 - numpy.abs(90-nlat_10[i]), latlon=True, color="white", linewidth=0, marker="o",zorder=8)
    
    ax.set_title("Magnetic north pole at year {0:4.1f}".format(year), fontdict={"fontsize" : "large"})
    
    fig.tight_layout()
    fig.savefig("magpole_frames/frame_{0:04d}".format(i))
    pyplot.close(fig)

In [26]:
fig, axes = pyplot.subplots(4,1, figsize=(15.46,11.56))

ax1, ax2, ax3, ax4 = axes

years = numpy.array(sha.years)

#intensity, intensity_dev = numpy.loadtxt("../data/pole_drift/intensity.txt").T*(6371000**3/10**-7*10**-6)
ax1.plot(years[years >= -2000], avdm[years >= -2000], color="blue")
ax1.fill_between(years[years >= -2000],
                 avdm[years >= -2000]-avdm_dev[years >= -2000],
                 avdm[years >= -2000]+avdm_dev[years >= -2000], color="blue", alpha=0.07)

ax1.set_xbound(-2000, 1900)
ax1.set_ylabel("Average VDM (Am$^{2}$)")

gallet_jerks=[-800, 200, 800, 1400]
pqpd_peaks=[-1450, -450, -150]

for ax in axes:
    for jerk in gallet_jerks:
        ax.plot([jerk, jerk], [-9999, 10**25], "r--", scaley=False, scalex=False)
    for jerk in pqpd_peaks:
        ax.plot([jerk, jerk], [-9999, 10**25], "b--", scaley=False, scalex=False)

pyplot.show(fig)

In [ ]:
s, z, r, phi, s_dev, z_dev, r_dev, phi_dev = numpy.loadtxt("../data/pole_drift/szrphi.txt").T
ax2.plot(years[years >= -2000], r[years >= -2000], color="blue", label="r")
ax2.plot(years[years >= -2000], z[years >= -2000], color="magenta", label="z")
ax2.plot(years[years >= -2000], s[years >= -2000], color="green", label="s")
ax2.fill_between(years[years >= -2000],
                 r[years >= -2000]-r_dev[years >= -2000],
                 r[years >= -2000]+r_dev[years >= -2000], color="blue", alpha=0.07)
ax2.fill_between(years[years >= -2000],
                 z[years >= -2000]-z_dev[years >= -2000],
                 z[years >= -2000]+z_dev[years >= -2000], color="magenta", alpha=0.07)
ax2.fill_between(years[years >= -2000],
                 s[years >= -2000]-s_dev[years >= -2000],
                 s[years >= -2000]+s_dev[years >= -2000], color="green", alpha=0.07)

ax2.set_xbound(-2000, 1900)
ax2.set_ylabel("Eccentric dipole location (km)")
ax2.legend(loc="lower center", fontsize="small", ncol=3)

pqpd, pqpd_dev = numpy.loadtxt("../data/pole_drift/pqpd.txt").T
ax3.plot(years[years >= -2000], pqpd[years >= -2000], color="blue")
ax3.fill_between(years[years >= -2000],
                 pqpd[years >= -2000]-pqpd_dev[years >= -2000],
                 pqpd[years >= -2000]+pqpd_dev[years >= -2000], color="blue", alpha=0.07)

ax3.set_xbound(-2000, 1900)
ax3.set_ylabel("$<Q>/<D>$")

curv_10 = curvature(nlon_10, nlat_10)
curv_1 = curvature(nlon_1, nlat_1)
ax4.plot(years[years >= -2000][:-1], curv_1[:-1], "b-", label="$l=1$")
ax4.plot(years[years >= -2000][:-1], curv_10[:-1], "k-", label="$l=10$")
ax4.set_ylabel("Pole drift curvature (º Years$^{-2})$")
ax4.set_xbound(-2000, 1900)
ax4.set_xlabel("Years AD")
ax4.legend(loc="upper right", fontsize="small")

gallet_jerks=[-800, 200, 800, 1400]
pqpd_peaks=[-1450, -450, -150]

for ax in axes:
    for jerk in gallet_jerks:
        ax.plot([jerk, jerk], [-9999, 10**25], "r--", scaley=False, scalex=False)
    for jerk in pqpd_peaks:
        ax.plot([jerk, jerk], [-9999, 10**25], "b--", scaley=False, scalex=False)

In [68]:
def curvature(lon, lat):
    #change this to use an equal-distance projection so tangent vector is accurate
    
    slon = lon % 360
    slat = 90 - numpy.abs(90-lat)

    fake_map = Basemap(projection='npaeqd',boundinglat=0,lon_0=0,resolution='l')

    xx, yy = fake_map(slon, slat)
    difx, dify = numpy.gradient(xx)/6371000, numpy.gradient(yy)/6371000
    tan_len = numpy.sqrt(difx**2 + dify**2)/50

    curvature = (numpy.gradient(difx)**2 + numpy.gradient(dify)**2)/tan_len
    return curvature

In [16]:
fig, ax = pyplot.subplots()

years = numpy.array(sha.years)

ax.plot(years[years >= -2000], avdm[years >= -2000]/10**22)

pyplot.show(fig)